# Summarize FY24 and FY25 Department of Justice Awards from USA Spending Data

This notebook analyzes contract data from the USA Spending database for fiscal years 2024 and 2025 for Department of Justice awards. The workflow:

1. [Data Acquisition](#data-acquisition): Connects to the USASpending API to fetch contract data for Department of Justice for fiscal years 2024 and 2025.
2. [Data Download](#data-download): Automatically downloads ZIP files containing contract data based on URLs returned by the API.
3. [Data Extraction](#data-extraction): Extracts the downloaded ZIP files into a data folder.
4. [Data Processing](#data-processing): Reads and combines all CSV files into a single dataframe.
5. [Data Selection](#data-selection): Creates a focused dataset by extracting key columns from the main dataframe, reducing the original 297-column dataset to 29 essential fields:
    - Contract identifiers: `award_id_piid`, `modification_number`
    - Timing information: `action_date_fiscal_year`, `last_modified_date`
    - Awarding entity details:
      - Sub-agency information: `awarding_sub_agency_code`, `awarding_sub_agency_name`
      - Office information: `awarding_office_code`, `awarding_office_name`
    - Funding entity details:
      - Sub-agency information: `funding_sub_agency_code`, `funding_sub_agency_name`
      - Office information: `funding_office_code`, `funding_office_name`
    - Recipient details: 
      - Direct recipient information: `recipient_uei`, `recipient_name`, `recipient_name_raw`
      - Parent organization information: `recipient_parent_uei`, `recipient_parent_name`, `recipient_parent_name_raw`
    - Contract information: `transaction_description`
    - Contract categorization: `product_or_service_code`, `product_or_service_code_description`
    - Financial metrics: `federal_action_obligation`, `total_dollars_obligated`, `total_outlayed_amount_for_overall_award`, `base_and_exercised_options_value`, `current_total_value_of_award`, `base_and_all_options_value`, `potential_total_value_of_award`
    - Reference information: `usaspending_permalink`
6. [Analytical Views](#analytical-views): Creates eight different summary views of the data:
    - [Award ID Analysis](#award-id-analysis): Groups contracts by award ID to identify the largest contracts by total potential value
    - [Recipient Analysis](#recipient-analysis): Groups by recipient UEI to analyze which vendors received the most contract dollars
    - [Parent Organization Analysis](#parent-organization-analysis): Aggregates by parent organizations to understand overall organizational contract awards
    - [Product/Service Analysis](#productservice-analysis): Identifies which types of products and services received the most funding
    - [Awarding Sub-agency Analysis](#awarding-sub-agency-analysis): Shows which specific bureaus or divisions within DOJ are allocating the most contract dollars
    - [Awarding Office Analysis](#awarding-office-analysis): Identifies which specific DOJ procurement offices are handling the largest contract values
    - [Funding Sub-agency Analysis](#funding-office-sub-agency-analysis): Examines which bureaus or divisions within DOJ are providing the funding for contracts
    - [Funding Office Analysis](#funding-office-analysis): Provides a granular breakdown of which specific DOJ offices are financially supporting contract awards
7. [Data Export](#data-export): Saves all views to a multi-sheet Excel file for further analysis and sharing

The analysis reveals significant contract awards, with some individual contracts valued at billions of dollars. The data includes detailed information about recipients, contract modifications, and the types of products and services being procured by the Department of Justice.

In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
import requests
import zipfile
import os

# supress warnings
import warnings
warnings.filterwarnings('ignore')

### Data Acquisition

Connects to the USASpending API to fetch contract data for Department of Justice for fiscal years 2024 and 2025.

In [2]:
# define a list of fiscal years
fys = [2024, 2025]
# initiate an empty list to store the data
data = []

# for each fiscal year, download the list of monthly files based on the request below
for fy in fys:
    url = "https://api.usaspending.gov/api/v2/bulk_download/list_monthly_files"
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    payload = {
        "agency": "17", # Department of Justice code
        "fiscal_year": str(fy),
        "type": "contracts"
    }
    response = requests.post(url, headers=headers, json=payload)
    # add the first object in the "monthly_files" array to the data list
    monthly_files = response.json()
    if "monthly_files" in monthly_files and len(monthly_files["monthly_files"]) > 0:
        data.append(monthly_files["monthly_files"][0])

    
# load the data list as a pandas dataframe
monthly_files_df = pd.DataFrame(data)
monthly_files_df.head()

,fiscal_year,agency_name,agency_acronym,type,updated_date,file_name,url
0,2024,Department of Justice,DOJ,contracts,2025-02-06,FY2024_015_Contracts_Full_20250206.zip,https://files.usaspending.gov/award_data_archi...
1,2025,Department of Justice,DOJ,contracts,2025-02-06,FY2025_015_Contracts_Full_20250206.zip,https://files.usaspending.gov/award_data_archi...


### Data Download

Automatically downloads ZIP files containing contract data based on URLs returned by the API.

In [3]:
# for each url in the "url" column, download the zip file and save it to the "data" folder
for index, row in monthly_files_df.iterrows():
    url = row["url"]
    filename = url.split("/")[-1]
    r = requests.get(url, allow_redirects=True)
    # save the zip file to the "data" folder
    # if the "data" folder does not exist, create it
    try:
        os.mkdir("data")
    except FileExistsError:
        pass

    with open(f"data/{filename}", "wb") as f:
        f.write(r.content)

### Data Extraction

Extracts the downloaded ZIP files into a `/data` folder.

In [4]:
# extract the zip files in the "data" folder
for index, row in monthly_files_df.iterrows():
    filename = row["url"].split("/")[-1]
    with zipfile.ZipFile(f"data/{filename}", "r") as zip_ref:
        zip_ref.extractall("data")

### Data Processing

Reads and combines all CSV files into a single dataframe (`df`).

In [5]:
# read each csv file in the "data" folder and concatenate them into a single dataframe

# Create a list to store each CSV file's dataframe
dfs = []

# Iterate through each csv file in the "data" folder
for file in os.listdir("data"):
    if file.endswith(".csv"):
        df = pd.read_csv(f"data/{file}")
        dfs.append(df)

# Combine the list of dataframes into one dataframe
df = pd.concat(dfs, ignore_index=True)

df.head()

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,...,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,initial_report_date,last_modified_date
0,1501_4732_15JTAX22F00000053_P00008_GS00F156GA_0,CONT_AWD_15JTAX22F00000053_1501_GS00F156GA_4732,15JTAX22F00000053,P00008,0.0,4732.0,FEDERAL ACQUISITION SERVICE,GS00F156GA,PA0019,149841.82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_15J...,2024-09-26,2024-09-30
1,1501_1501_15JPSS23F00000433_P00002_15JPSS20G00...,CONT_AWD_15JPSS23F00000433_1501_15JPSS20G00000...,15JPSS23F00000433,P00002,0.0,1501.0,"OFFICES, BOARDS AND DIVISIONS",15JPSS20G00000334,0,-2561.54,...,200000.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_15J...,2024-09-23,2024-09-30
2,1501_8000_15JPSS24F00000983_0_NNG15SD91B_0,CONT_AWD_15JPSS24F00000983_1501_NNG15SD91B_8000,15JPSS24F00000983,0,0.0,8000.0,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,NNG15SD91B,11,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_15J...,2024-09-30,2024-10-01
3,1544_-NONE-_15M10224PA4700495_0_-NONE-_0,CONT_AWD_15M10224PA4700495_1544_-NONE-_-NONE-,15M10224PA4700495,0,0.0,NaN,NaN,NaN,NaN,54725.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_15M...,2024-09-30,2024-09-30
4,1542_1542_15UASH24F00000851_0_15UC0C21D00001511_0,CONT_AWD_15UASH24F00000851_1542_15UC0C21D00001...,15UASH24F00000851,0,0.0,1542.0,"FEDERAL PRISON INDUSTRIES, INC.",15UC0C21D00001511,0,43452.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_15U...,2024-10-01,2024-10-01


In [6]:
# save the combined dataframe to a CSV file

df.to_csv(f"data/015_Contracts_Full.csv", index=False)
print(f"Data saved to data/015_Contracts_Full.csv")

Data saved to data/015_Contracts_Full.csv


### Data Selection

This cell creates a focused dataset (`df_narrow`) by extracting key columns from the main dataframe, reducing the original 297-column dataset to 29 essential fields:

- Contract identifiers: `award_id_piid`, `modification_number`
- Timing information: `action_date_fiscal_year`, `last_modified_date`
- Awarding entity details:
    - Sub-agency information: `awarding_sub_agency_code`, `awarding_sub_agency_name`
    - Office information: `awarding_office_code`, `awarding_office_name`
- Funding entity details:
    - Sub-agency information: `funding_sub_agency_code`, `funding_sub_agency_name`
    - Office information: `funding_office_code`, `funding_office_name`
- Recipient details: 
    - Direct recipient information: `recipient_uei`, `recipient_name`, `recipient_name_raw`
    - Parent organization information: `recipient_parent_uei`, `recipient_parent_name`, `recipient_parent_name_raw`
- Contract information: `transaction_description`
- Contract categorization: `product_or_service_code`, `product_or_service_code_description`
- Financial metrics: 
    - Current action values: `federal_action_obligation`
    - Cumulative values: `total_dollars_obligated`, `total_outlayed_amount_for_overall_award`
    - Contract ceiling values: `base_and_exercised_options_value`, `current_total_value_of_award`, `base_and_all_options_value`, `potential_total_value_of_award`
- Reference information: `usaspending_permalink` - direct link to record on USA Spending website

The data is sorted by award ID and modification date (descending) to group related contract actions together with the most recent modifications appearing first.


In [7]:
# select key columns from the dataframe
df_narrow = df[['award_id_piid', 'modification_number', 'action_date_fiscal_year', 'awarding_sub_agency_code', 'awarding_sub_agency_name', 'awarding_office_code', 'awarding_office_name', 'funding_sub_agency_code', 'funding_sub_agency_name', 'funding_office_code', 'funding_office_name', 'recipient_uei', 'recipient_name', 'recipient_name_raw', 'recipient_parent_uei', 'recipient_parent_name', 'recipient_parent_name_raw', 'transaction_description', 'product_or_service_code_description', 'product_or_service_code', 'federal_action_obligation', 'total_dollars_obligated', 'total_outlayed_amount_for_overall_award', 'base_and_exercised_options_value', 'current_total_value_of_award', 'base_and_all_options_value', 'potential_total_value_of_award', 'usaspending_permalink', 'last_modified_date']]

# Change column type to datetime64[ns] for column: 'last_modified_date'
df_narrow = df_narrow.astype({'last_modified_date': 'datetime64[ns]'})

# sort the dataframe by award_id_piid and last_modified_date in descending order
df_narrow = df_narrow.sort_values(['award_id_piid', 'last_modified_date'], ascending=[True, False])

# display a sample of the data: the first 100 rows, filtered by product_or_service_code containing "7E20" and sorted by potential_total_value_of_award in descending order
df_narrow[df_narrow['product_or_service_code'].str.contains("7E20", regex=False, na=False, case=False)].sort_values(['potential_total_value_of_award'], ascending=[False]).head(100)

,award_id_piid,modification_number,action_date_fiscal_year,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,...,product_or_service_code,federal_action_obligation,total_dollars_obligated,total_outlayed_amount_for_overall_award,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,usaspending_permalink,last_modified_date
161679,15F06724A0000311,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+10,8.600000e+10,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18
30903,15F06724A0000311,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+10,8.600000e+10,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18
30448,15F06724A0000335,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+09,8.600000e+09,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18
161087,15F06724A0000343,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+09,8.600000e+09,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18
161041,15F06724A0000340,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+09,8.600000e+09,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161186,15F06724A0000299,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+09,8.600000e+09,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-19
30410,15F06724A0000299,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+09,8.600000e+09,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-19
161021,15F06724A0000298,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+09,8.600000e+09,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18
30245,15F06724A0000298,0,2024,1549,Federal Bureau of Investigation,15F067,DIVISION 1200,1549,Federal Bureau of Investigation,15F067,...,7E20,0.0,0.0,NaN,NaN,NaN,8.600000e+09,8.600000e+09,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18


### Analytical Views

A shared function `get_most_recent_potential_value()` creates all summary views by finding the most recent record for each group (based on modification date), extracting the potential award value, counting modifications, formatting currency values, and sorting results with highest values first.

The eight analytical views generated using this function are:

1. **Award ID Analysis**: Groups by individual contract identifiers to identify the largest contracts by total potential value, revealing which specific agreements represent the most significant financial commitments.
2. **Recipient Analysis**: Aggregates by vendor UEI to determine which contractors received the largest dollar values, showing direct recipients of federal funds.
3. **Parent Organization Analysis**: Consolidates data by parent organizations to reveal the true scale of contracts flowing to major corporate entities that may operate through multiple subsidiaries.
4. **Product/Service Analysis**: Groups by product or service codes to identify which categories of goods and services represent the largest areas of spending.
5. **Awarding Sub-agency Analysis**: Groups by awarding sub-agency to identify which specific bureaus or divisions within DOJ agencies are allocating the most contract dollars.
6. **Awarding Office Analysis**: Analyzes spending by individual awarding offices, revealing which specific DOJ procurement offices are handling the largest contract values.
7. **Funding Sub-agency Analysis**: Examines which bureaus or divisions within DOJ are providing the funding for contracts, which may differ from the awarding entities.
8. **Funding Office Analysis**: Provides a granular breakdown of which specific DOJ offices are financially supporting contract awards, offering insight into the source of funds.

Each analysis includes the following metrics:
- Latest potential value of award
- Fiscal year
- Most recent modification date
- Number of contract modifications

This multi-dimensional approach provides comprehensive visibility into contract spending patterns across the Department of Justice.

In [8]:
def get_most_recent_potential_value(df, group_field, additional_fields=None):
    """
    Groups the dataframe by a provided field and returns the potential value of the award 
    for the most recent record in each group, with optional additional fields.
    
    Parameters:
    df (DataFrame): The input dataframe
    group_field (str): The field to group by
    additional_fields (list, optional): Additional fields to include in the result
    
    Returns:
    DataFrame: Grouped dataframe with the most recent values
    """
    # Default fields to include in the result
    default_fields = [group_field, 'modification_number', 'transaction_description', 'action_date_fiscal_year', 'potential_total_value_of_award', 'row_count', 'usaspending_permalink', 'last_modified_date']
    
    # Add additional fields if provided
    all_fields = default_fields.copy()
    if additional_fields:
        for field in additional_fields:
            if field in df.columns and field not in all_fields:
                all_fields.append(field)
    
    # Group by group_field and get max last_modified_date
    df_grouped_group_field = df.groupby(group_field)['last_modified_date'].max().reset_index()
    
    # Merge to get potential_total_value_of_award for max last_modified_date
    df_grouped_group_field_max_last_modified = pd.merge(df, df_grouped_group_field, on=[group_field, 'last_modified_date'], how='inner')
    
    # Group by group_field to get the number of rows
    row_counts = df_grouped_group_field_max_last_modified.groupby(group_field).size().reset_index(name='row_count')
    
    # Merge row counts back to the main DataFrame
    df_grouped_group_field_max_last_modified = pd.merge(df_grouped_group_field_max_last_modified, row_counts, on=group_field, how='inner')
    
    # Select relevant columns
    df_grouped_group_field_max_last_modified = df_grouped_group_field_max_last_modified[all_fields]
    
    # Sort by potential_total_value_of_award before formatting
    df_grouped_group_field_max_last_modified = df_grouped_group_field_max_last_modified.sort_values(
        by='potential_total_value_of_award', ascending=False)

    # Format the potential_total_value_of_award column as currency
    df_grouped_group_field_max_last_modified['potential_total_value_of_award'] = df_grouped_group_field_max_last_modified['potential_total_value_of_award'].map('${:,.2f}'.format)
    
    # Format the last_modified_date column as date
    df_grouped_group_field_max_last_modified['last_modified_date'] = df_grouped_group_field_max_last_modified['last_modified_date'].dt.strftime('%Y-%m-%d')
    
    return df_grouped_group_field_max_last_modified

#### Award ID Analysis

In [9]:
# 1. group by award_id_piid and max(last_modified) and return max(potential_total_value_of_award) count(rows)
df_grouped_award_id = get_most_recent_potential_value(df_narrow, 'award_id_piid')
df_grouped_award_id.head()

,award_id_piid,modification_number,transaction_description,action_date_fiscal_year,potential_total_value_of_award,row_count,usaspending_permalink,last_modified_date
97080,15F06724A0000311,0,ITSSS-2 BPA MASTER AWARD,2024,"$86,000,000,000.00",2,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18
97081,15F06724A0000311,0,ITSSS-2 BPA MASTER AWARD,2024,"$86,000,000,000.00",2,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18
93397,15F06719F0001923,P00016,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18
93395,15F06719F0001923,P00016,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18
93394,15F06719F0001923,P00015,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18


#### Recipient Analysis

In [10]:
# 2. group by recipient_uei and max(last_modified) and return max(potential_total_value_of_award) count(rows)
df_grouped_recipient_uei = get_most_recent_potential_value(df_narrow, 'recipient_uei', ['recipient_name', 'recipient_name_raw', 'recipient_parent_name', 'recipient_parent_name_raw'])
df_grouped_recipient_uei.head()

,recipient_uei,modification_number,transaction_description,action_date_fiscal_year,potential_total_value_of_award,row_count,usaspending_permalink,last_modified_date,recipient_name,recipient_name_raw,recipient_parent_name,recipient_parent_name_raw
9669,QGJNGLBLVKY6,P00016,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC",CHENEGA INTEGRATED SECURITY SOLUTIONS LLC
9670,QGJNGLBLVKY6,P00015,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC",CHENEGA INTEGRATED SECURITY SOLUTIONS LLC
9671,QGJNGLBLVKY6,P00016,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC",CHENEGA INTEGRATED SECURITY SOLUTIONS LLC
9668,QGJNGLBLVKY6,P00015,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC",CHENEGA INTEGRATED SECURITY SOLUTIONS LLC
10772,EH2ULQUQ2KG9,0,ITSSS-2 MASTER BPA,2024,"$8,600,000,000.00",2,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18,COLSA CORP,COLSA CORP,COLSA CORP,COLSA CORPORATION


#### Parent Organization Analysis

In [11]:
# 3. group by recipient_parent_uei and max(last_modified) and return max(potential_total_value_of_award) count(rows)
df_grouped_recipient_parent_uei = get_most_recent_potential_value(df_narrow, 'recipient_parent_uei', ['recipient_parent_name', 'recipient_parent_name_raw', 'recipient_name', 'recipient_name_raw'])
df_grouped_recipient_parent_uei.head()

,recipient_parent_uei,modification_number,transaction_description,action_date_fiscal_year,potential_total_value_of_award,row_count,usaspending_permalink,last_modified_date,recipient_parent_name,recipient_parent_name_raw,recipient_name,recipient_name_raw
10136,QGJNGLBLVKY6,P00015,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC",CHENEGA INTEGRATED SECURITY SOLUTIONS LLC,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC"
10137,QGJNGLBLVKY6,P00016,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC",CHENEGA INTEGRATED SECURITY SOLUTIONS LLC,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC"
10139,QGJNGLBLVKY6,P00016,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC",CHENEGA INTEGRATED SECURITY SOLUTIONS LLC,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC"
10138,QGJNGLBLVKY6,P00015,SECD SUPPORT SERVICES - BPA CALL 3,2025,"$50,481,562,555.57",4,https://www.usaspending.gov/award/CONT_AWD_15F...,2024-11-18,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC",CHENEGA INTEGRATED SECURITY SOLUTIONS LLC,"CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC","CHENEGA INTEGRATED SECURITY SOLUTIONS, LLC"
11380,JPHFV985CTT2,0,ITSSS-2 BPA MASTER AWARD,2024,"$8,600,000,000.00",2,https://www.usaspending.gov/award/CONT_IDV_15F...,2024-09-18,"BASTION ANALYTICS, LLC",BASTION ANALYTICS LLC,"BASTION ANALYTICS, LLC","BASTION ANALYTICS, LLC"


#### Product/Service Analysis

In [12]:
# 4. group by product_or_service_code and max(last_modified) and return max(potential_total_value_of_award) count(rows)

df_grouped_product_or_service_code = get_most_recent_potential_value(df_narrow, 'product_or_service_code', ['product_or_service_code_description'])
df_grouped_product_or_service_code.head()

,product_or_service_code,modification_number,transaction_description,action_date_fiscal_year,potential_total_value_of_award,row_count,usaspending_permalink,last_modified_date,product_or_service_code_description
2251,D399,P00037,"NOVATION FOR NAME CHANGE TO AT&T ENTERPRISES, LLC",2025,"$4,934,000,000.00",2,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-01-31,IT AND TELECOM- OTHER IT AND TELECOMMUNICATIONS
2250,D399,P00037,"NOVATION FOR NAME CHANGE TO AT&T ENTERPRISES, LLC",2025,"$4,934,000,000.00",2,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-01-31,IT AND TELECOM- OTHER IT AND TELECOMMUNICATIONS
1907,C1AA,P00003,ACADEMIC ZONE DESIGN-BUILD PROJECT. MOD P00003...,2025,"$512,709,360.00",2,https://www.usaspending.gov/award/CONT_AWD_15F...,2025-01-27,ARCHITECT AND ENGINEERING- CONSTRUCTION: OFFIC...
1906,C1AA,P00003,ACADEMIC ZONE DESIGN-BUILD PROJECT. MOD P00003...,2025,"$512,709,360.00",2,https://www.usaspending.gov/award/CONT_AWD_15F...,2025-01-27,ARCHITECT AND ENGINEERING- CONSTRUCTION: OFFIC...
1060,Y1FF,P00005,"MPR-005 FOR: $56,116.00 MPR-008 FOR: $33,310....",2025,"$461,497,000.00",2,https://www.usaspending.gov/award/CONT_AWD_15B...,2025-01-23,CONSTRUCTION OF PENAL FACILITIES


#### Awarding Sub-agency Analysis

In [13]:
# 6. group by awarding_sub_agency_code and max(last_modified) and return max(potential_total_value_of_award) count(rows)

df_grouped_awarding_sub_agency_code = get_most_recent_potential_value(df_narrow, 'awarding_sub_agency_code', ['awarding_sub_agency_name'])
df_grouped_awarding_sub_agency_code.head()

,awarding_sub_agency_code,modification_number,transaction_description,action_date_fiscal_year,potential_total_value_of_award,row_count,usaspending_permalink,last_modified_date,awarding_sub_agency_name
590,1501,P00045,DOJ OCIO CSS ICAM SUPPORT,2025,"$398,835,404.29",104,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,"Offices, Boards and Divisions"
591,1501,P00045,DOJ OCIO CSS ICAM SUPPORT,2025,"$398,835,404.29",104,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,"Offices, Boards and Divisions"
578,1501,P00049,"CSS POLICY, AUDIT, AND ADMINISTRATION BPA CALL",2025,"$79,703,479.20",104,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,"Offices, Boards and Divisions"
579,1501,P00049,"CSS POLICY, AUDIT, AND ADMINISTRATION BPA CALL",2025,"$79,703,479.20",104,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,"Offices, Boards and Divisions"
736,1544,P00030,BUSINESS CASE & BUSINESS PROCESS ANALYSIS,2025,"$57,188,017.95",148,https://www.usaspending.gov/award/CONT_AWD_15M...,2025-02-03,U.S. Marshals Service


#### Awarding Office Analysis

In [14]:
# 7. group by awarding_office_code and max(last_modified) and return max(potential_total_value_of_award) count(rows)

df_grouped_awarding_office_code = get_most_recent_potential_value(df_narrow, 'awarding_office_code', ['awarding_office_name'])
df_grouped_awarding_office_code.head()

,awarding_office_code,modification_number,transaction_description,action_date_fiscal_year,potential_total_value_of_award,row_count,usaspending_permalink,last_modified_date,awarding_office_name
1877,15M400,P00223,COMPREHENSIVE DETENTION SERVICES CENTRAL AZ,2024,"$4,386,714,974.50",2,https://www.usaspending.gov/award/CONT_IDV_DJJ...,2025-01-29,"U.S. DEPT OF JUSTICE, USMS"
1876,15M400,P00223,COMPREHENSIVE DETENTION SERVICES CENTRAL AZ,2024,"$4,386,714,974.50",2,https://www.usaspending.gov/award/CONT_IDV_DJJ...,2025-01-29,"U.S. DEPT OF JUSTICE, USMS"
1406,15JPSS,P00045,DOJ OCIO CSS ICAM SUPPORT,2025,"$398,835,404.29",24,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,JMD-PROCUREMENT SERVICES STAFF
1407,15JPSS,P00045,DOJ OCIO CSS ICAM SUPPORT,2025,"$398,835,404.29",24,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,JMD-PROCUREMENT SERVICES STAFF
1393,15JPSS,P00049,"CSS POLICY, AUDIT, AND ADMINISTRATION BPA CALL",2025,"$79,703,479.20",24,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,JMD-PROCUREMENT SERVICES STAFF


#### Funding Office Sub-agency Analysis

In [15]:
# 8. group by funding_sub_agency_code and max(last_modified) and return max(potential_total_value_of_award) count(rows)

df_grouped_funding_sub_agency_code = get_most_recent_potential_value(df_narrow, 'funding_sub_agency_code', ['funding_sub_agency_name'])
df_grouped_funding_sub_agency_code.head()

,funding_sub_agency_code,modification_number,transaction_description,action_date_fiscal_year,potential_total_value_of_award,row_count,usaspending_permalink,last_modified_date,funding_sub_agency_name
601,1501,P00045,DOJ OCIO CSS ICAM SUPPORT,2025,"$398,835,404.29",101,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,"Offices, Boards and Divisions"
600,1501,P00045,DOJ OCIO CSS ICAM SUPPORT,2025,"$398,835,404.29",101,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,"Offices, Boards and Divisions"
586,1501,P00049,"CSS POLICY, AUDIT, AND ADMINISTRATION BPA CALL",2025,"$79,703,479.20",101,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,"Offices, Boards and Divisions"
587,1501,P00049,"CSS POLICY, AUDIT, AND ADMINISTRATION BPA CALL",2025,"$79,703,479.20",101,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-02-03,"Offices, Boards and Divisions"
748,1544,P00030,BUSINESS CASE & BUSINESS PROCESS ANALYSIS,2025,"$57,188,017.95",130,https://www.usaspending.gov/award/CONT_AWD_15M...,2025-02-03,U.S. Marshals Service


#### Funding Office Analysis

In [16]:
# 9. group by funding_office_code and max(last_modified) and return max(potential_total_value_of_award) count(rows)

df_grouped_funding_office_code = get_most_recent_potential_value(df_narrow, 'funding_office_code', ['funding_office_name'])
df_grouped_funding_office_code.head()

,funding_office_code,modification_number,transaction_description,action_date_fiscal_year,potential_total_value_of_award,row_count,usaspending_permalink,last_modified_date,funding_office_name
1589,15JSDS,P00037,"NOVATION FOR NAME CHANGE TO AT&T ENTERPRISES, LLC",2025,"$4,934,000,000.00",4,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-01-31,SERVICE DELIVERY STAFF (JMD)
1588,15JSDS,P00037,"NOVATION FOR NAME CHANGE TO AT&T ENTERPRISES, LLC",2025,"$4,934,000,000.00",4,https://www.usaspending.gov/award/CONT_AWD_15J...,2025-01-31,SERVICE DELIVERY STAFF (JMD)
3069,15M400,P00223,COMPREHENSIVE DETENTION SERVICES CENTRAL AZ,2024,"$4,386,714,974.50",2,https://www.usaspending.gov/award/CONT_IDV_DJJ...,2025-01-29,"U.S. DEPT OF JUSTICE, USMS"
3068,15M400,P00223,COMPREHENSIVE DETENTION SERVICES CENTRAL AZ,2024,"$4,386,714,974.50",2,https://www.usaspending.gov/award/CONT_IDV_DJJ...,2025-01-29,"U.S. DEPT OF JUSTICE, USMS"
1592,15JAFM,P00047,ASSET FORFEITURE ADMINISTRATIVE SERVICES,2025,"$1,311,230,136.00",2,https://www.usaspending.gov/award/CONT_IDV_15J...,2025-01-24,ASSET FORFEITURE MGMT STAFF (JMD)


### Data Export

Saves all views to a multi-sheet Excel file for further analysis and sharing. The resulting Excel file "USA_Spending_Summary_Analysis.xlsx" contains:

1. **Raw Data**: Complete dataset with all 29 key columns for detailed record-level analysis
2. **Summary by Award ID**: Contracts aggregated by individual award ID, revealing largest contracts by total potential value
3. **Summary by Recipient UEI**: Aggregation by vendor identifier showing which contractors received the most funding
4. **Summary by Parent Recipient UEI**: Consolidation by parent organizations showing the true scale of corporate contract awards
5. **Summary by PSC**: Product/Service Code summary revealing which categories of goods and services received the most funding
6. **Summary by Awarding Sub-agency**: Breakdown showing which specific bureaus or divisions within DOJ are allocating the most contract dollars
7. **Summary by Awarding Office**: Detailed view of which specific DOJ procurement offices are handling the largest contract values
8. **Summary by Funding Sub-agency**: Analysis of which bureaus or divisions within DOJ are providing the funding for contracts (as opposed to awarding them)
9. **Summary by Funding Office**: Granular breakdown of which specific DOJ offices are financially supporting contract awards

Each summary sheet presents data sorted by potential total value (highest first), allowing quick identification of the most significant contracts, recipients, parent organizations, product/service categories, and both awarding and funding entities across Department of Justice spending.

In [17]:
# export each dataframe to a separate sheet within an excel file
# use a relative path rather than an absolute path
with pd.ExcelWriter(r'USA_Spending_Summary_Analysis.xlsx') as writer:  
    df_narrow.to_excel(writer, sheet_name='raw_data')
    df_grouped_award_id.to_excel(writer, sheet_name='Summary by Award')
    df_grouped_recipient_uei.to_excel(writer, sheet_name='Summary by Recipient')
    df_grouped_recipient_parent_uei.to_excel(writer, sheet_name='Summary by Parent Recipient')
    df_grouped_product_or_service_code.to_excel(writer, sheet_name='Summary by PSC')
    df_grouped_awarding_sub_agency_code.to_excel(writer, sheet_name='Summary by Awarding Sub Agency')
    df_grouped_awarding_office_code.to_excel(writer, sheet_name='Summary by Awarding Office')
    df_grouped_funding_sub_agency_code.to_excel(writer, sheet_name='Summary by Funding Sub Agency')
    df_grouped_funding_office_code.to_excel(writer, sheet_name='Summary by Funding Office')

print("Excel file created successfully.")

Excel file created successfully.
